In [1]:
!pip install sqlite3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('test.db')
 
conn.execute('''
CREATE TABLE campaign_comms_user_level (
campaign_id INTEGER,
user_id INTEGER,
date_sent INTEGER,
opened_email INTEGER,
clicked_on_link INTEGER,
visited_landing_page INTEGER,
converted INTEGER
);''')

In [3]:
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (183946, 68324, 20180705, 1, 1, 0, 0);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (183946, 65715, 20180712, 1, 0, 0, 0);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421564, 46546, 20180702, 1, 1, 1, 1);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (654654, 86451, 20180801, 0, 0, 0, 0);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421564, 65158, 20180807, 1, 0, 0, 0);''')
conn.commit()

In [4]:
cursor = conn.execute(''' SELECT * FROM campaign_comms_user_level ''')
for row in cursor:
  print(row)

(183946, 68324, 20180705, 1, 1, 0, 0)
(183946, 65715, 20180712, 1, 0, 0, 0)
(421564, 46546, 20180702, 1, 1, 1, 1)
(654654, 86451, 20180801, 0, 0, 0, 0)
(421564, 65158, 20180807, 1, 0, 0, 0)


a.	Calculate a “funnel” per campaign (= ordered milestones with proportion of successes in each milestone based on the successes in the previous milestone).

In [7]:
#a.	Calculate a “funnel” per campaign (= ordered milestones with proportion of successes in each milestone based on the successes in the previous milestone).
cursor = conn.execute('''SELECT 

    campaign_id, 

    COUNT (CASE WHEN campaign_comms_user_level.opened_email=1 THEN 1 END) AS opened_email, 

    COUNT (CASE WHEN  campaign_comms_user_level.clicked_on_link=1 THEN 1 END) AS clicked_on_link, 

    COUNT (CASE WHEN campaign_comms_user_level.visited_landing_page=1 THEN 1 END) AS visited_landing_page,
    
    COUNT (CASE WHEN campaign_comms_user_level.converted=1 THEN 1 END) AS converted 

FROM 

    campaign_comms_user_level 

WHERE 

    campaign_id IS NOT NULL 

GROUP BY 

    campaign_id;''')
for row in cursor:
  print(row)

# Expected output
# (183946, 2, 1, 0, 0)
# (421564, 2, 1, 1, 1)
# (654654, 0, 0, 0, 0)

(183946, 2, 1, 0, 0)
(421564, 2, 1, 1, 1)
(654654, 0, 0, 0, 0)


There are numbers of events for each campaign id

In [8]:


cursor = conn.execute('''SELECT 

    campaign_id, 

    1.0*COUNT (CASE WHEN campaign_comms_user_level.opened_email THEN 1 END)/ COUNT (CASE WHEN campaign_comms_user_level.date_sent THEN 1 ELSE 1 END) AS p_opened_email, 

    1.0*COUNT (CASE WHEN  campaign_comms_user_level.clicked_on_link THEN 1 END)/ COUNT (CASE WHEN campaign_comms_user_level.opened_email THEN 1 ELSE 1 END) AS p_clicked_on_link, 

    1.0*COUNT (CASE WHEN campaign_comms_user_level.visited_landing_page THEN 1  END)/ COUNT (CASE WHEN campaign_comms_user_level.clicked_on_link THEN 1 ELSE 1 END) AS p_visited_landing_page,
    
    1.0*COUNT (CASE WHEN campaign_comms_user_level.converted THEN 1 END)/ COUNT (CASE WHEN campaign_comms_user_level.visited_landing_page THEN 1 ELSE 1 END) AS p_converted 

FROM 

    campaign_comms_user_level 

WHERE 

    campaign_id IS NOT NULL 

GROUP BY 

    campaign_id;''')
for row in cursor:
  print(row)

# Expected output
# (183946, 1.0, 0.5, 0.0, 0.0)
# (421564, 1.0, 0.5, 0.5, 0.5)
# (654654, 0.0, 0.0, 0.0, 0.0)

(183946, 1.0, 0.5, 0.0, 0.0)
(421564, 1.0, 0.5, 0.5, 0.5)
(654654, 0.0, 0.0, 0.0, 0.0)


There are ratios of numbers of successful each events divided on number of previos milestone event

In [13]:
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421564, 65150, 20180807, 1, 0, 0, 1);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421564, 65150, 20180807, 1, 0, 0, 1);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421564, 65150, 20180807, 1, 0, 0, 1);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421564, 65150, 20180807, 1, 0, 0, 1);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421564, 65150, 20180807, 1, 0, 0, 1);''')
conn.commit()

In [14]:
#b. Find the most successful campaign in terms of
# i) proportion of opened (among sent) emails
cursor = conn.execute('''SELECT
	campaign_id,
    1.0*COUNT (CASE WHEN campaign_comms_user_level.opened_email THEN 1 END)/ COUNT (CASE WHEN campaign_comms_user_level.date_sent THEN 1 ELSE 1 END) AS ratio_opened_sent_email 
FROM 

    campaign_comms_user_level 

WHERE 

    campaign_id IS NOT NULL 

GROUP BY 

    campaign_id
ORDER BY ratio_opened_sent_email DESC
LIMIT 1;''');
for row in cursor:
  print(row)
#Expected output 
#(421564, 1.0)

(421564, 1.0)


In [15]:
#b. Find the most successful campaign in terms of
# ii) proportion of conversions among emails opened.
cursor = conn.execute('''SELECT
	campaign_id,
    1.0*COUNT (CASE WHEN campaign_comms_user_level.converted THEN 1 END)/ COUNT (CASE WHEN campaign_comms_user_level.opened_email THEN 1 ELSE 1 END) AS ratio_converted_opened_email 
FROM 

    campaign_comms_user_level 

WHERE 

    campaign_id IS NOT NULL 

GROUP BY 

    campaign_id
ORDER BY ratio_converted_opened_email DESC
LIMIT 1;''');
for row in cursor:
  print(row)
#Expected output -  6 conversions /7 sent emails from campaign id 421564
#(421564, 0.8571428571428571)

(421564, 0.8571428571428571)


In [17]:
cursor = conn.execute(''' SELECT * FROM campaign_comms_user_level WHERE campaign_id=421564''')
for row in cursor:
  print(row)

(421564, 46546, 20180702, 1, 1, 1, 1)
(421564, 65158, 20180807, 1, 0, 0, 0)
(421564, 65150, 20180807, 1, 0, 0, 1)
(421564, 65150, 20180807, 1, 0, 0, 1)
(421564, 65150, 20180807, 1, 0, 0, 1)
(421564, 65150, 20180807, 1, 0, 0, 1)
(421564, 65150, 20180807, 1, 0, 0, 1)


c. Find all users who were contacted with at least 5 campaigns and
i) converted in more than 75% of cases
ii) never converted.

In [18]:
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421560, 65150, 20180807, 1, 0, 0, 1);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421561, 65150, 20180807, 1, 0, 0, 1);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421562, 65150, 20180807, 1, 0, 0, 1);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421563, 65150, 20180807, 1, 0, 0, 1);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421565, 65150, 20180807, 1, 0, 0, 1);''')
conn.commit()

In [19]:
# Find all users who were contacted with at least 5 campaigns i) converted in more than 75% of cases

cursor = conn.execute('''SELECT
		user_id
FROM 
    campaign_comms_user_level 
GROUP BY
		user_id
HAVING	
		COUNT(DISTINCT(campaign_id))>=5 AND (COUNT(converted)/COUNT(*))>0.75;''');
for row in cursor:
  print(row)

#Expected output 
#(65150,)

(65150,)


In [20]:
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421560, 6515012, 20180807, 1, 0, 0, 0);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421561, 6515012, 20180807, 1, 0, 0, 0);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421562, 6515012, 20180807, 1, 0, 0, 0);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421563, 6515012, 20180807, 1, 0, 0, 0);''')
conn.execute('''INSERT INTO campaign_comms_user_level VALUES (421565, 6515012, 20180807, 1, 0, 0, 0);''')
conn.commit()

In [21]:
# Find all users who were contacted with at least 5 campaigns and ii) never converted.

cursor = conn.execute('''SELECT
		user_id
FROM 
    		campaign_comms_user_level 
GROUP BY
				user_id
HAVING	
				COUNT(DISTINCT(campaign_id))>=5 AND SUM(converted)=0;
	''');
for row in cursor:
  print(row)

	#Expected output
	# (6515012,)

(6515012,)


2.	Regarding the metrics calculated in the previous task, is there any methodological issue / something we need to take care of? How would you assess the quality of the data stored in campaign_comms_user_level?
 
Answer:

There are could be incorrectly calculated metrics in case of moving on next event will be triggered further milestones skipping previous steps, for example if email functionality provides opportunity without opening email to click on  link in content of message (on a title of message)
So that means in case of succesful final step(converted=1) but previous actions in data filled as 0, replace them by 1.

The table campaign_comms_user_levels seems that probably was created by pivot function,they were beed aggregated from original data comms_user. The wide format of storing in the format does not consume so much size and readable instead of long original logs. 



3.	Write an SQL script which creates the above table (campaign_comms_user_level) using only the information from the table below (called campaign_comms): 
 
date 	campaign_id 	user_id 	event 

 20180722 	183946 	657464654 	opened_email 

20180801 	421564 	6874654651 	converted 

 
Where event can have the following values:  
•	‘email_sent’, ‘email_opened’, ‘clicked_on_link’, ‘visited_landing_page’, ‘converted’ 


In [22]:
cursor = conn.execute('''CREATE TABLE campaign_comms(
  date INTEGER,
  campaign_id INTEGER,
  user_id INTEGER,
  event VARCHAR(20)
);''')
for row in cursor:
  print(row)

In [23]:
cursor = conn.execute('''INSERT INTO campaign_comms VALUES (20180722, 	183946, 	657464654, 'opened_email');''');
cursor = conn.execute('''INSERT INTO campaign_comms VALUES (20180801, 421564, 	6874654651, 'converted');''')
cursor = conn.execute(''' SELECT * FROM campaign_comms;''')
for row in cursor:
  print(row)


(20180722, 183946, 657464654, 'opened_email')
(20180801, 421564, 6874654651, 'converted')


In [25]:

cursor = conn.execute('''CREATE TABLE campaign_comms_user_level1 AS 
	SELECT date AS date_sent, campaign_id, user_id, 
      SUM (CASE WHEN event = 'opened_email' THEN 1 ELSE 0 END) AS opened_email,
      SUM (CASE WHEN event  = 'clicked_on_link' THEN 1 ELSE 0 END) AS clicked_on_link,
      SUM (CASE WHEN event  = 'visited_landing_page' THEN 1 ELSE 0 END) AS visited_landing_page,
      SUM (CASE WHEN event  = 'converted' THEN 1 ELSE 0 END) AS converted
     FROM campaign_comms
     GROUP BY user_id;''')




In [26]:
cursor = conn.execute('''SELECT * FROM campaign_comms_user_level1;''')
for row in cursor:
  print(row)

# Expected output
# (20180722, 183946, 657464654, 1, 0, 0, 0)
# (20180801, 421564, 6874654651, 0, 0, 0, 1)

(20180722, 183946, 657464654, 1, 0, 0, 0)
(20180801, 421564, 6874654651, 0, 0, 0, 1)
